# Import packages

In [1]:
import pyforest
from tqdm import tqdm, tqdm_notebook
import spacy
import scispacy  # https://allenai.github.io/scispacy/
spacy.prefer_gpu()
from spacy import displacy
from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)
import uuid
from colorama import Fore

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

c:\programdata\anaconda3\envs\graph\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Load data from CDC and start processing for NER

In [2]:
df=pd.read_csv('D:/GitHub/COVID-19/data/CDC/All_Articles_Excel.csv',encoding='utf-8', 
               usecols=["Abstract","Title","Author","Journal/Publisher","Year","Name of Database"], 
               dtype={'Abstract': str,'Title': str,'Journal/Publisher': str,'Author': str,'Name of Database': str,'Year': str})

<IPython.core.display.Javascript object>

In [3]:
#Count the articls we have
df.shape

(53351, 6)

In [4]:
# remove articles that do no have an Abstract since that is what we are using for NER
df = df[df['Abstract'].notna()]
# df.shape

In [5]:
df.head()

,Author,Title,Abstract,Year,Journal/Publisher,Name of Database
0,"Zurayk, R",Pandemic and food security: a view from the Gl...,This paper offers a view of COVID-19's specifi...,2020,"Journal of Agriculture, Food Systems and Commu...",CAB Abstracts
1,"Zou, YZ, YiShan; Cao, Jun",Suggestions for standardizing the process of m...,"At the end of 2019, the epidemic of coronaviru...",2020,Chinese Journal of Parasitology and Parasitic ...,CAB Abstracts
2,"Zou, XL, Xiaolei; Zhou, Renxue; Yi, Jiahui; Xu...",Composition containing sodium chloride and suc...,The present invention relates to a composition...,2020,NaN,CAS
3,"Zhu, TX, A.; Bai, X.; He, Y.; Zhang, H.",[Effect of convalescent plasma and immunoglobu...,OBJECTIVE: To systematically review evidence f...,2020,Zhonghua wei zhong bing ji jiu yi xue,PubMed
4,"Zhu, JZ, Zhimei; Li, Hongyuan; Pan, Ji; Pang, ...",CT imaging features of 4121 patients with COVI...,ObjectiveWe systematically reviewed the comput...,2020,Journal of Medical Virology,Agricultural & Environmental Science Collectio...


In [6]:
df['uuid'] = df.apply(lambda x: uuid.uuid4(), axis=1)

In [7]:
print(df.Abstract)

0        This paper offers a view of COVID-19's specifi...
1        At the end of 2019, the epidemic of coronaviru...
2        The present invention relates to a composition...
3        OBJECTIVE: To systematically review evidence f...
4        ObjectiveWe systematically reviewed the comput...
                               ...                        
47839    Diante da emergência de agentes com capacidade...
47840    [Introdução]: Essa é a primeira edição deste d...
47849    [Antecedentes]: Este documento resume as orien...
47852    Incluye una colección de recursos sobre el nue...
47854    As of 6am Geneva time this morning, China has ...
Name: Abstract, Length: 29061, dtype: object


## Visualize Bio-Medical entities 

In [8]:
# This function will allow us to randomly color the bio-medical entities that teh spacy NER model finds.
# THis is mainly for display purposes
import random 

def get_entity_options(random_colors=False):
    """
    generating color options for visualizing the named entities
    """
    def color_generator(number_of_colors):
        color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
                 for i in range(number_of_colors)]
        return color

    entities = ["GGP", "SO", "TAXON", "CHEBI", "GO", "CL", 
                "DNA", "CELL_TYPE", "CELL_LINE", "RNA", "PROTEIN",
                "DISEASE", "CHEMICAL",
                "CANCER", "ORGAN", "TISSUE", "ORGANISM", "CELL", 
                "AMINO_ACID", "GENE_OR_GENE_PRODUCT", "SIMPLE_CHEMICAL", 
                "ANATOMICAL_SYSTEM", "IMMATERIAL_ANATOMICAL_ENTITY", "MULTI-TISSUE_STRUCTURE",
                "DEVELOPING_ANATOMICAL_STRUCTURE", "ORGANISM_SUBDIVISION", "CELLULAR_COMPONENT"]
    
    colors = {"ENT":"#82E0AA"}
    
    if random_colors:
        color = color_generator(len(entities))
        for i in range(len(entities)):
            colors[entities[i]] = color[i]
    else:
        entities_cat_1 = {"GGP":"#F9E79F", "SO":"#F7DC6F", "TAXON":"#F4D03F", "CHEBI":"#FAD7A0", "GO":"#F8C471", "CL":"#F5B041"}
        entities_cat_2 = {"DNA":"#82E0AA", "CELL_TYPE":"#AED6F1", "CELL_LINE":"#E8DAEF", "RNA":"#82E0AA", "PROTEIN":"#82E0AA"}
        entities_cat_3 = {"DISEASE":"#D7BDE2", "CHEMICAL":"#D2B4DE"}
        entities_cat_4 = {"CANCER":"#ABEBC6", "ORGAN":"#82E0AA", "TISSUE":"#A9DFBF", "ORGANISM":"#A2D9CE", "CELL":"#76D7C4", "AMINO_ACID":"#85C1E9", "GENE_OR_GENE_PRODUCT":"#AED6F1", "SIMPLE_CHEMICAL":"#76D7C4", "ANATOMICAL_SYSTEM":"#82E0AA", "IMMATERIAL_ANATOMICAL_ENTITY":"#A2D9CE", "MULTI-TISSUE_STRUCTURE":"#85C1E9", "DEVELOPING_ANATOMICAL_STRUCTURE":"#A9DFBF", "ORGANISM_SUBDIVISION":"#58D68D", "CELLULAR_COMPONENT":"#7FB3D5"}

        entities_cats = [entities_cat_1, entities_cat_2, entities_cat_3, entities_cat_4]
        for item in entities_cats:
            colors = {**colors, **item}
    
    options = {"ents": entities, "colors": colors}
    
    return options

In [9]:
# Text block for NER testing
text_test = str("The ongoing Coronavirus Disease 2019 (COVID-19) pandemic caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) signals an urgent need for an expansion in treatment options In this study, we investigated the anti-SARS-CoV-2 activities of 22 antiviral agents with known broad-spectrum antiviral activities against coronaviruses and/or other viruses They were first evaluated in our primary screening in VeroE6 cells and then the most potent anti-SARS-CoV-2 antiviral agents were further evaluated using viral antigen expression, viral load reduction, and plaque reduction assays In addition to remdesivir, lopinavir, and chloroquine, our primary screening additionally identified types I and II recombinant interferons, 25-hydroxycholesterol, and AM580 as the most potent anti-SARS-CoV-2 agents among the 22 antiviral agents Betaferon (interferon-&amp;beta;1b) exhibited the most potent anti-SARS-CoV-2 activity in viral antigen expression, viral load reduction, and plaque reduction assays among the recombinant interferons The lipogenesis modulators 25-hydroxycholesterol and AM580 exhibited EC50 at low micromolar levels and selectivity indices of &amp;gt;10 0 Combinational use of these host-based antiviral agents with virus-based antivirals to target different processes of the SARS-CoV-2 replication cycle should be evaluated in animal models and/or clinical trials")

In [10]:
from spacy import displacy
nlp = spacy.load("en_ner_bionlp13cg_md")
# nlp = spacy.load("en_ner_bc5cdr_md")
# nlp = spacy.load("en_ner_jnlpba_md")
# nlp = spacy.load("en_ner_craft_md")
doc = nlp(text_test)
displacy.render(doc, style="ent", options=get_entity_options(random_colors=True))

## Lets apply the NER model to the Abstract's and output that to a dataframe that is ready for import into Neo4j as node's

In [11]:
getattr(tqdm, '_instances', {}).clear()  # ⬅ clear the progress
spacy.prefer_gpu()
nlp = spacy.load("en_ner_bionlp13cg_md")
entities=[]
uuid = str(df.uuid)
for i in tqdm(df['Abstract'].tolist(),bar_format="{l_bar}%s{bar}%s{r_bar}" % (Fore.GREEN, Fore.RESET)):
    doc = nlp(i)
    for entity in doc.ents:
        entities.append((i, entity.text, entity.label_, uuid))

100%|████████████████████████████████████████████████████████████████████████████| 29061/29061 [17:14<00:00, 28.09it/s]


In [12]:
df_ner = pd.DataFrame.from_records(entities, columns =['Abstract','ent_name', 'ent_type', 'uuid'])

<IPython.core.display.Javascript object>

In [13]:
df_ner.head()

,Abstract,ent_name,ent_type,uuid
0,This paper offers a view of COVID-19's specifi...,COVID-19's,GENE_OR_GENE_PRODUCT,0 6798d439-f19f-41fa-9406-16a11186fda7\...
1,This paper offers a view of COVID-19's specifi...,COVID-19,GENE_OR_GENE_PRODUCT,0 6798d439-f19f-41fa-9406-16a11186fda7\...
2,"At the end of 2019, the epidemic of coronaviru...",coronavirus disease 2019,ORGANISM,0 6798d439-f19f-41fa-9406-16a11186fda7\...
3,"At the end of 2019, the epidemic of coronaviru...",COVID-19,GENE_OR_GENE_PRODUCT,0 6798d439-f19f-41fa-9406-16a11186fda7\...
4,"At the end of 2019, the epidemic of coronaviru...",patients,ORGANISM,0 6798d439-f19f-41fa-9406-16a11186fda7\...


### Let's remove duplicate entities and their types from each abstract since we care abou the realtionships between articles

In [14]:
#remove duplicates
df_ner.drop_duplicates(subset=['Abstract', 'ent_name', 'ent_type'], keep='first')

,Abstract,ent_name,ent_type,uuid
0,This paper offers a view of COVID-19's specifi...,COVID-19's,GENE_OR_GENE_PRODUCT,0 6798d439-f19f-41fa-9406-16a11186fda7\...
1,This paper offers a view of COVID-19's specifi...,COVID-19,GENE_OR_GENE_PRODUCT,0 6798d439-f19f-41fa-9406-16a11186fda7\...
2,"At the end of 2019, the epidemic of coronaviru...",coronavirus disease 2019,ORGANISM,0 6798d439-f19f-41fa-9406-16a11186fda7\...
3,"At the end of 2019, the epidemic of coronaviru...",COVID-19,GENE_OR_GENE_PRODUCT,0 6798d439-f19f-41fa-9406-16a11186fda7\...
4,"At the end of 2019, the epidemic of coronaviru...",patients,ORGANISM,0 6798d439-f19f-41fa-9406-16a11186fda7\...
...,...,...,...,...
326479,Incluye una colección de recursos sobre el nue...,información,ORGANISM_SUBSTANCE,0 6798d439-f19f-41fa-9406-16a11186fda7\...
326480,Incluye una colección de recursos sobre el nue...,países,SIMPLE_CHEMICAL,0 6798d439-f19f-41fa-9406-16a11186fda7\...
326481,Incluye una colección de recursos sobre el nue...,vídeos,CELLULAR_COMPONENT,0 6798d439-f19f-41fa-9406-16a11186fda7\...
326482,"As of 6am Geneva time this morning, China has ...",Geneva,ORGANISM,0 6798d439-f19f-41fa-9406-16a11186fda7\...


## Output the files so I can clean and join them later

In [15]:
df_ner.to_csv("ner_out_uuid.csv", sep=',',index=False)

In [16]:
df.to_csv("abstract_out_uuid.csv", sep=',',index=False)

## ToDo

- Add UUID to entity output for matching later
- Filter entities so if it appears more than once it only shows up in output
- For filtered entities, keep a count of the number of times it was mentioned
- Extract the authors individually 
- in some Journals, the author named first is primary but in other journals it is the last author
- See if we can find if that author is cited in the article